# Take-home Assignment by Eero Antikainen

To start with, we can just run the script as a whole:

In [ ]:
from main import main

main()

## Part 1: ETL

At first I glanced at the csv's to see what's in them.

customers.csv looked mostly ok, transactions.csv had a good amount of missing values.

For this amount of data and the complexity of processes I planned to do, I decided SQLite is more than fine.

To begin with, I thought that I'd load the data pretty much as is to the database for easy querying.

So after setting up the database, the only transformations I decided to do here is to turn missing values into None instead of empty strings:

In [2]:
def transform_customers(raw_data: list[tuple]) -> list[tuple]:
  return [
    (
      int(customer_id) if customer_id else None,
      country if country else None,
      signup_date if signup_date else None,
      email if email else None
    )
    for customer_id, country, signup_date, email in raw_data
  ]

This transformation allowed me to insert the missing values as NULL easily.

## Part 2: Feature engineering

I proceeded to do some anomaly checks to understand the problems of the data better.

Results displayed from the pipeline:

```2026-02-05 21:22:17,924 - INFO - Found 30 NULL values in transactions table column customer_id
2026-02-05 21:22:17,924 - INFO - Total value: 1410.28 EUR
2026-02-05 21:22:17,925 - INFO - Found 24644 NULL values in transactions table column currency
2026-02-05 21:22:17,925 - INFO - Found 20220 NULL values in transactions table column category
2026-02-05 21:22:17,929 - INFO - Total value: 887961.98 EUR
2026-02-05 21:22:18,014 - INFO - 85435 transactions found where timestamp precedes users signup
2026-02-05 21:22:18,033 - INFO - Total value: 3717602.34 EUR
2026-02-05 21:22:18,083 - INFO - Found 2000 duplicated transaction ids
2026-02-05 21:22:18,084 - INFO - Getting deeper comparison
2026-02-05 21:22:18,213 - INFO - Found 2000 duplicated transactions
2026-02-05 21:22:18,214 - INFO - All the duplicates are exact matches and can be filtered via 'GROUP BY transaction_id'

The first part was straightforward query of NULL values for each column in both tables and logging results, when found.

I also logged the total values transferred to Euros (except for NULL values in currency column)

Turns out the customers data was fine and transactions had more inconsistencies.

I decided to assume that category was not required information, but others I would flag for closer inspection.

Both tables had time related column in them so I wanted to check for inconsistencies there.

And I found 3.7 million Eur worth of transactions by users before their signup. More flags.

The last anomaly check was found by accident when I was looking at `average_days_between_transactions` (now `purchase_interval`) data and at the top of the list there was a user with 2 transactions, 0 days in between, and identical sum. This inspired me to check possible duplicate id's, after which I did a deeper comparison. I suppose one could've gone just with the deeper comparison straight away, but I thought there might have been transactions that share only the transaction id because of a race condition or something similar.

With all this insight, I cleaned the data:

In [ ]:
import sqlite3

from utils.logging_config import logger


def clean_transactions(conn: sqlite3.Connection) -> None:
  logger.info("Cleaning the dataset after anomaly analysis")
  cur = conn.cursor()
  cur.execute(
    """
      CREATE VIEW clean_transactions AS
      SELECT
        transaction_id,
        transactions.customer_id customer_id,
        amount,
        currency,
        timestamp,
        category
      FROM transactions
      INNER JOIN customers
      ON transactions.customer_id = customers.customer_id
      WHERE currency IS NOT NULL
        AND transactions.customer_id IS NOT NULL
        AND transactions.timestamp >= customers.signup_date
      GROUP BY transaction_id
    """
  )

### Insights

During the process I rotated the data a handful of ways to find something interesting. Here are some insights from the views I ended up in:

#### Customers with the lowest purchase interval

```Showing top 10 of purchase_interval ascending
customer_id  transactions  avg_spent  max_spent  min_spent  total_spent    first_transaction     last_transaction  purchase_interval
       1714             2     121.14     179.30      62.97       242.27  2020-11-18 23:04:00  2020-11-19 13:32:00                0.6
       3980             2      97.74     118.38      77.09       195.47  2020-12-09 04:53:00  2020-12-10 05:43:00                1.0
       2327             4     130.17     184.07      56.93       520.66  2020-11-10 08:58:00  2020-11-14 01:39:00                1.2
       4391             3     119.97     183.59      87.04       359.92  2020-12-08 07:19:00  2020-12-11 21:24:00                1.8
       3014             2     101.25     109.01      93.48       202.49  2020-11-09 15:09:00  2020-11-11 11:24:00                1.8
        136             2     115.91     130.87     100.95       231.82  2020-11-05 15:54:00  2020-11-08 04:02:00                2.5
       3637             2     110.53     113.18     107.87       221.05  2020-10-20 10:16:00  2020-10-23 12:06:00                3.1
        117             2     169.10     187.12     151.07       338.19  2020-11-26 03:28:00  2020-11-29 13:05:00                3.4
       3587             3     126.94     204.91      77.08       380.83  2020-09-16 13:25:00  2020-09-23 20:25:00                3.6
       4280             5     139.75     197.68     104.88       698.75  2020-10-18 08:45:00  2020-11-02 17:48:00                3.8

Here we see the lowest purchase intervals, the highest transaction frequencies. It seems that none of them have many transactions. They had a couple of transactions in a short period of time. The amounts spent are quite regular as well.

#### The biggest total_spent

```Showing top 10 of total_spent descending
customer_id  transactions  avg_spent  max_spent  min_spent  total_spent    first_transaction     last_transaction  purchase_interval
       1725            36     105.74     210.19      12.36      3806.59  2020-01-29 06:53:00  2020-12-06 07:54:00                8.9
       1655            34     106.63     175.60      39.19      3625.30  2020-01-04 16:48:00  2020-11-02 03:57:00                9.2
       3407            33     106.00     215.77      46.34      3498.12  2020-02-10 19:49:00  2020-12-01 11:36:00                9.2
       4201            30     114.62     186.04      36.94      3438.49  2020-01-20 01:14:00  2020-12-12 12:00:00               11.3
       3893            32     103.84     199.95       7.11      3322.99  2020-01-03 05:56:00  2020-12-09 11:33:00               11.0
       2022            31     104.70     191.65      13.16      3245.72  2020-01-10 00:20:00  2020-11-27 08:59:00               10.7
        676            29     110.72     174.90      13.18      3210.90  2020-01-02 02:36:00  2020-12-06 16:14:00               12.1
         72            29     109.64     195.97       3.81      3179.51  2020-01-02 18:46:00  2020-11-28 15:37:00               11.8
        107            32      99.36     199.06      14.87      3179.47  2020-01-06 00:52:00  2020-11-28 06:06:00               10.6
       2934            32      98.58     219.59     -23.16      3154.43  2020-01-03 08:33:00  2020-11-22 11:59:00               10.5

Their intervals vary between 1 and 2 weeks, but they all have almost 30 transactions. The average spending amounts are marginally even lower than the visitors from the first table. They have transactions from the beginning of the time scope until the end. This table is naturally very similar to the amount of transactions:

#### The most transactions

```Showing top 10 of transactions descending
customer_id  transactions  avg_spent  max_spent  min_spent  total_spent    first_transaction     last_transaction  purchase_interval
       1725            36     105.74     210.19      12.36      3806.59  2020-01-29 06:53:00  2020-12-06 07:54:00                8.9
       1655            34     106.63     175.60      39.19      3625.30  2020-01-04 16:48:00  2020-11-02 03:57:00                9.2
       3407            33     106.00     215.77      46.34      3498.12  2020-02-10 19:49:00  2020-12-01 11:36:00                9.2
       3893            32     103.84     199.95       7.11      3322.99  2020-01-03 05:56:00  2020-12-09 11:33:00               11.0
        107            32      99.36     199.06      14.87      3179.47  2020-01-06 00:52:00  2020-11-28 06:06:00               10.6
       2934            32      98.58     219.59     -23.16      3154.43  2020-01-03 08:33:00  2020-11-22 11:59:00               10.5
       3576            31     100.56     187.77      22.48      3117.44  2020-01-02 03:25:00  2020-12-03 15:40:00               11.2
       2022            31     104.70     191.65      13.16      3245.72  2020-01-10 00:20:00  2020-11-27 08:59:00               10.7
       2355            30     102.22     192.98      35.53      3066.74  2020-01-06 09:04:00  2020-12-11 14:19:00               11.7
       4295            30     104.32     175.42      23.48      3129.68  2020-01-20 01:51:00  2020-12-01 09:29:00               10.9

Identical top 3 and the rest consist almost of the same customers too. The biggest one time spenders are bringing in a decent amount of cash flow as well:

#### The biggest max_spent

```Showing top 10 of max_spent descending
customer_id  transactions  avg_spent  max_spent  min_spent  total_spent    first_transaction     last_transaction  purchase_interval
       1877            19     115.55     263.11      38.07      2195.37  2020-01-04 09:35:00  2020-12-06 18:04:00               18.7
       3195            21      98.78     253.31      35.79      2074.42  2020-01-29 00:51:00  2020-12-07 12:44:00               15.7
        489            18     111.34     252.85      56.55      2004.13  2020-01-05 12:43:00  2020-11-27 05:32:00               19.2
       3374            13     113.94     251.14      36.38      1481.26  2020-01-01 12:20:00  2020-11-30 14:27:00               27.8
       2933            12     105.69     250.88      43.32      1268.22  2020-01-30 20:41:00  2020-11-23 03:24:00               27.0
       4754            20     106.88     249.66      40.50      2137.55  2020-01-05 16:21:00  2020-11-06 06:36:00               16.1
       2564            17     122.73     245.24      72.47      2086.42  2020-01-16 05:50:00  2020-11-22 18:25:00               19.5
       3907            15     129.22     239.18      35.82      1938.36  2020-05-04 17:26:00  2020-11-10 22:48:00               13.6
        766            18     104.01     236.87      39.03      1872.22  2020-03-22 11:18:00  2020-12-11 23:04:00               15.6
       1517            25     102.82     236.75      20.41      2570.44  2020-02-20 20:00:00  2020-11-13 02:38:00               11.1

And finally, the churning risk:

#### The longest since their last transaction

```Showing top 10 of days_since_last_transaction descending
customer_id  transactions  total_spent  purchase_interval  days_since_last_transaction
       2502             9       894.24               21.6                        185.0
       2214             9      1135.11               26.4                        153.0
       1080            12      1163.22               18.2                        151.0
       1041            12      1031.66               18.4                        146.0
       1147            14      1237.58               16.2                        144.0
        172            20      1890.06               12.2                        125.0
       4906            19      1549.62               13.5                        121.0
       2905            14      1299.31               18.9                        120.0
       4911            15      1653.80               14.9                        117.0
       1881            11      1134.18               19.6                        111.0

I suppose one should prioritize these customers by their `total_spent` amount to keep the most valuable customers, if prioritization is needed.

### Downstream use

Both dataframes are saved in their entirety as `customer_features.csv` and `churning_risks.csv` in the `./data` directory. From there they could be easily fed to a BI tool for example.

## Part 3: LLM pipeline

Given the minimal document set (11 lines total), I decided to mock the RAG pipeline to demonstrate the architecture rather than implement actual embeddings.

In [ ]:
from random import random

from utils.file_reader import get_filenames


def vectorize(query: str = "This is a query.") -> list[float]:
  embedding = []
  logger.info(f"Vectorizing query: {query}")
  files = get_filenames()
  for _ in files:
    embedding.append(random())
  return embedding

So I start by "vectorizing" the query. The `vectorize` function returns a list of floats. The list has the same amount of elements, as there are documents in the documents folder. The values are random values between 0 and 1.

With the embedding, we can retrieve with "vector search" or "cosine similarity". Instead of implementing that, or even vectorizing the source material, decided to treat the random values of the vector as the cosine similarity score for the respective documents.

In [ ]:
from utils.file_reader import read_doc

def retrieve(vector: list[float], top_k: int, threshold: float) -> dict:
 
  context = {}
  filenames = get_filenames()
  scores = {}
  # Give each filename their mock cosine similarity value
  for idx, filename in enumerate(filenames):
    scores[filename] = vector[idx]
  # Sort the filenames by said values
  top_results = sorted(scores, key=scores.get, reverse=True)
  # Get top-k results that are over the threshold value 
  for result in top_results[:top_k]:
    if scores[result] > threshold:
      context[result] = read_doc(f"./data/documents/{result}")
  return context

With the context (if found), I can produce the "response", which I also mocked:

In [ ]:
def generate_response(context) -> str:
  response = "Hi! I'm your happy assistant!\n"
  if context:
    response += "I'm basing my helpful response to the following contents:\n"
    for item in context:
      response +=f"\n{item}\n{context[item]}\n"
  else:
    response += "This time we did not find what you were looking for."
  
  return response

## Part 4: Serve

I built a simple API with emphasis on the simple. It has 2 endpoints in the end. One for "asking questions about the documents" and the other for uploading more documents.

In [ ]:
from fastapi import FastAPI
from utils.filename_validation import validate_filename

app = FastAPI()

@app.post("/ask")
def ask(query: str, top_k: int, threshold: float):
  embedding = vectorize(query)
  context = retrieve(embedding, top_k, threshold)
  response = generate_response(context)
  return {"query": query, "response": response}

@app.post("/upload")
def upload(filename: str, content: str):
  validated_filename = validate_filename(filename)
  if type(validated_filename) == str:
    with open(f"./data/documents/{validated_filename}", "w") as file:
      file.write(content)
    return { "status": 200 }
  else:
    return { "status": 403, "details": validated_filename["reason"] }

I added a basic validation for filenames, so only .txt's can get through and the user can't overwrite existing files:

In [ ]:
from utils.file_reader import get_filenames

ValidationObject = dict[str, str | bool]

def validate_filename(filename) -> str | ValidationObject:
  if filename.endswith('.txt'):
    if '.' in filename[:-4]:
      return { "success": False, "reason": "Invalid filename" }
  else:
    if '.' in filename:
      return { "success": False, "reason": "Invalid filename" }
    filename += '.txt'

  if filename in get_filenames():
    return { "success": False, "reason": "Duplicate filename" }
  
  return filename

## Summary

This project demonstrates a complete data pipeline:

1. **ETL**: Extracted CSVs, transformed types, loaded to SQLite
2. **Data Quality**: Identified ~85k problematic transactions, 2k duplicates, created clean view
3. **Feature Engineering**: Built customer metrics (spend, frequency, churn risk)
4. **LLM Pipeline**: Mock RAG demonstrating retrieve → augment → generate flow
5. **API**: FastAPI service with /ask and /upload endpoints

**Key insight**: Revenue correlates with transaction frequency, not transaction size. Customer retention is more valuable than pursuing large one-time sales.

**Output**: Clean feature tables exported to CSV, ready for BI tools or further analysis.